## Mean change in annual extreme heat day and warm nights

This notebook generates the text metadata files for the extreme heat exposure metric `mean change in annual extreme heat day` and `mean change in annual warm nights` metrics, using data from Cal-Adapt: Analytics Engine data. Because the AE data represents 200+ GB of data, metrics were calculated with a cluster in a high performance computing environment (i.e., a pcluster). Please see the processing script `climate_ae_extreme_heat_metrics.py` for full methodological process. 

### Step 1: Generate metadata

In [2]:
import pandas as pd
import os
import sys

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import upload_csv_aws, pull_csv_from_directory
from scripts.utils.write_metadata import append_metadata

In [ ]:
bucket_name = 'ca-climate-index'
aws_dir = '3_fair_data/index_data/climate_extreme_heat_warm_night_metric.csv'
folder = 'csv_folder'

pull_csv_from_directory(bucket_name, aws_dir, folder, search_zipped=False)

In [ ]:
# options here: climate_extreme_heat_hot_day_metric, climate_extreme_heat_warm_night_metric
df_in = pd.read_csv(r'csv_folder/climate_extreme_heat_warm_night_metric.csv') # make sure this is in the same folder!
df_in # check

In [5]:
# Move a specific column to the end of the DataFrame
column_to_move = 'mean_change_annual_warm_nights'  # Replace with the actual column name
columns = [col for col in df_in.columns if col != column_to_move]  # Keep all other columns
columns.append(column_to_move)  # Add the column to move to the end

# Reassign the DataFrame with the new column order
df_in = df_in[columns]

In [7]:
df_in.to_csv('climate_extreme_heat_warm_night_metric.csv', index=False)

In [8]:
@append_metadata
def extreme_heat_ae_data_process(df, export=False, export_filename=None, varname=''):
    '''
    Reduces the size of the initial daily raw temperature data in order to streamline compute time.
    Transforms the raw data into the following baseline metrics:
    * Mean change in annual number of warm nights
    * Mean change in annual number of extreme heat days
    
    Methods
    -------
    Metric is calculated by counting the total number of days greater than 98th historical percentile and taking the annual average.
    
    Parameters
    ----------
    df: pd.DataFrame
        Input data.
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI extreme heat metric to AWS
        True = will upload resulting df containing CAL CRAI extreme heat metric to AWS
    export_filename: string
        name of csv file to be uploaded to AWS
    varname: string
        Final metric name, for metadata generation
        
    Script
    ------
    Metric calculation: climate_ae_extreme_heat_metrics.py via pcluster run
    Example metric calculation for Los Angeles county: climate_ae_extreme_heat_metrics_example.ipynb
    Metadata generation: climate_ae_extreme_heat_metadata.ipynb
    
    Note
    ----
    Because the climate projections data is on the order of 2.4 TB in size, intermediary
    processed files are not produced for each stage of the metric calculation. All processing
    occurs in a single complete run in the notebook listed above.
    '''
        
    # calculate with 2°C WL
    print('Data transformation: raw projections data retrieved for warming level of 2.0°C, by manually subsetting based on GWL for parent GCM and calculating 30 year average.')
    print("Data transformation: dynamically-downscaled climate data subsetted for a-priori bias-corrected models.")

    # historical baseline
    print("Data transformation: historical baseline data retrieved for 1981-2010, averaging across models.")
    print("Data transformation: dynamically-downscaled climate data subsetted for a-priori bias-corrected models.")

    # calculate delta signal
    print("Data transformation: drop all singleton dimensions (scenario).")
    print("Data transformation: count total # of days above 98th historical percentile and calculate annual average.")
    print("Data transformation: delta signal calculated by taking difference between chronic (2.0°C) and historical baseline.")
    print("Data transformation: non-CA grid points removed from data.")

    # reprojection to census tracts
    print("Data transformation: data transformed from xarray dataset into pandas dataframe.")
    print("Data transformation: data reprojected from Lambert Conformal Conic CRS to CRS 3857.")
        
    # min-max standardization
    print("Data transformation: data min-max standardized with min_max_standardize function.")
    
    if export == True:
        bucket_name = 'ca-climate-index'
        directory = '3_fair_data/index_data'
        export_filename = [df]
        upload_csv_aws(export_filename, bucket_name, directory)

    if export == False:
        print(f'{df} uplaoded to AWS.')

    if os.path.exists(df):
        os.remove(df)

In [9]:
varname = 'climate_extreme_heat_warm_nights'
filename = 'climate_extreme_heat_warm_night_metric.csv'
extreme_heat_ae_data_process(filename, export=True, export_filename=None, varname='test')